In [ ]:
#first activate conda by $ conda activate /c/Users/darshan\ patel/Desktop/genai_first_project/env

In [1]:
import os
import json
import pandas as pd
import traceback

In [5]:
from langchain.chat_models import ChatOpenAI
import dotenv

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
KEY=os.getenv("OPENAI_API_KEY")


In [9]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-4o-mini",temperature=0.3)

C:\Users\darshan patel\AppData\Local\Temp\ipykernel_7404\1846689700.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-4o-mini",temperature=0.3)


In [12]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [13]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [14]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [15]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
    
)

In [16]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\darshan patel\AppData\Local\Temp\ipykernel_7404\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [17]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [18]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [19]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [27]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=False,)

In [28]:
#now creating a file data.txt which contains data to give llm
file_path=r"C:\Users\darshan patel\Desktop\genai_first_project\data.txt"


In [29]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [30]:
#to convert string which we pass to template1 (response _json)to convert it from string into original json
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [31]:
NUMBER=5 
SUBJECT="Machine Learning"
TONE="simple"

In [ ]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking
#not nessecary to write only gives you how many tokens are generated (that with line)
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)         #passing response_json as a json to template 1
        }
        )

In [ ]:
#we got response in string
quiz=response.get("quiz")


str

In [ ]:
#converting string into json
quiz=json.loads(quiz)


In [ ]:
#converting it into a python dictinary
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [41]:
quiz_table_data


[{'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Donald Hebb | b: Tom M. Mitchell | c: Arthur Samuel | d: Walter Pitts',
  'Correct': 'c'},
 {'MCQ': "What was the purpose of the 'learning machine' called Cybertron developed in the early 1960s?",
  'Choices': 'a: To play chess | b: To analyze sonar signals and speech patterns | c: To classify images | d: To teach humans',
  'Correct': 'b'},
 {'MCQ': 'What is a key objective of modern-day machine learning?',
  'Choices': 'a: To create video games | b: To classify data and make predictions | c: To replace human intelligence | d: To analyze historical data',
  'Correct': 'b'},
 {'MCQ': 'What did Tom M. Mitchell define as a condition for a computer program to learn?',
  'Choices': 'a: It must have a user interface | b: It must improve performance with experience | c: It must be able to play games | d: It must be connected to the internet',
  'Correct': 'b'},
 {'MCQ': 'Which book published in 1949 introduced a theoretical

In [42]:
quiz=pd.DataFrame(quiz_table_data)

In [43]:

quiz.to_csv("machinelearning.csv",index=False)